will add to README once this script is fully developed


## Running Inference_Example 

### without CLI flags
Edit the `DEFAULT_CONFIG` block near the top of `new_package_versions/Inference_Example.py` to set your start time, checkpoint location, output path, and variable list. Once those values are saved, launch the script without any flags (e.g., inside a batch script):

```bash
python Example_Code/inference/Inference_Example.py
```

Argparse will see no command-line inputs and will run entirely from `DEFAULT_CONFIG`, so you do not need to type flags each time.

### option: with CLI overrides
If you prefer to pass arguments explicitly, any flag you provide will override the corresponding entry in `DEFAULT_CONFIG`:

```bash
python Inference_Example.py \
  --start 2019-03-22T00:00:00 \
  --steps 12 \
  --init-data /path/to/Initialize_2019_03_22T00_nsteps12.nc \
  --checkpoint-dir /path/to/checkpoints \
  --checkpoint-name ckpt_mp0_epoch1.tar \
  --output /path/to/output/forecast.nc \
  --variables msl tcwv \
  --ema
```

### Outputs
The script loads the initial NetCDF, runs inference, and writes the final forecast step to the requested NetCDF path (creating parent directories if needed).


### Notes
- still need to setup code for generating initialization files, but for now will work with the initializtion files Becca provided, located in `/projectnb/eb-general/wade/sfno/inference_runs/Ian/Initialize_data`


In [1]:
from __future__ import annotations

import numpy as np
import torch
import xarray as xr
import earth2studio
from earth2studio.data import DataArrayFile
from earth2studio.io import ZarrBackend
from earth2studio.models.auto import Package

from earth2studio.run import deterministic
from earth2studio.models.px import SFNO

from utils import *

/projectnb/eb-general/wade/.conda/envs/earth2studio/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/projectnb/eb-general/wade/.conda/envs/earth2studio/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(earth2studio.__version__)
import makani
print(makani.__version__)

0.10.0rc0
0.2.0


In [3]:
@dataclass
class InferenceConfig:
    """Configuration container for running the inference script."""

    start: str = "2019-09-03T00:00:00" # fix that this has to be a different format from the initialize time string
    steps: int = 4
    init_data: Path = Path(
        "/projectnb/eb-general/wade/sfno/inference_runs/Ian/Initialize_data/Initialize_2019_09_03T00_nsteps4.nc"
    )
    checkpoint_dir: Path = Path(
        "/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/"
        "sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/"
    )  # TODO CHECK IF THIS PATH NEEDS TO HAVE ADDITIONAL DIRECTORY TO THE MULTISTEP OR NON MULTISTEP DIR + seed + training_checkpoints
    checkpoint_name: str = "ckpt_mp0_epoch1.tar"  # for now, the code will just grab best ckpt, not this one.
    output: Path = Path(
        "/projectnb/eb-general/wade/sfno/inference_runs/sandbox/best_ckpt_2019_09_04T00_nsteps4.nc"
    )  # todo make this automated to just take the directory, not the full path+name

    variables: list[str] | None = None
    ema: bool = False

DEFAULT_CONFIG = InferenceConfig()

In [4]:
def _normalize_valid_times(ds: xr.Dataset, final_timestamp: np.datetime64) -> xr.Dataset:
    """Align the earth2studio ``time`` and ``lead_time`` axes to usable timestamps.

    earth2studio returns forecasts with two axes: the initialization time (``time``)
    and the offsets from that initialization (``lead_time``). To mirror the legacy
    inference example, we compute the actual verification timestamps (``valid_time``),
    attach them as coordinates, and extract the requested final forecast hour.

    Parameters
    ----------
    ds : xr.Dataset
        Dataset returned by earth2studio after running inference.
    final_timestamp : np.datetime64
        Target forecast verification time to extract.
    """

    ds = ds.copy() # NEWLINE 
    ds["time"] = ds["time"].astype("datetime64[ns]") 
    base_time = ds["time"].values
    lead_timedelta = ds["lead_time"].values.astype("timedelta64[ns]")
    valid_timesteps = (base_time[:, None] + lead_timedelta[None, :]).flatten()

    # Preserve the initial time as metadata while collapsing the time dimension.
    initial_time = str(ds["time"].values[0])
    ds = ds.isel(time=0).drop_vars("time")
    ds.attrs["initial_time"] = initial_time

    ds = ds.rename({"lead_time": "valid_time"})
    ds = ds.assign_coords(valid_time=(("valid_time",), valid_timesteps))

    if final_timestamp not in ds["valid_time"].values:
        raise ValueError(
            f"Requested final datetime {final_timestamp} not found in forecast valid times: {ds['valid_time'].values!r}"
        )

    return ds.sel(valid_time=[final_timestamp])

In [45]:
def _load_model(package_dir: Path, 
                # checkpoint_name: str, ema: bool
        ) -> torch.nn.Module:
    model_package = Package(str(package_dir), cache=False)
    return SFNO.load_model(model_package, 
    # checkpoint_name=checkpoint_name, EMA=ema
    )

In [46]:
def _run_inference(
    start_datetime: datetime,
    n_steps: int,
    checkpoint_dir: Path,
    checkpoint_name: str,
    init_data_path: Path,
    output_path: Path,
    variables: Iterable[str] | None,
    ema: bool,
) -> None:
    if not init_data_path.exists():
        raise FileNotFoundError(
            f"Initial data file {init_data_path} not found. Generate it before running inference."
        )

    initial_data = DataArrayFile(str(init_data_path))
    model = _load_model(checkpoint_dir, 
    # checkpoint_name,  ema
    )
    # Since earth2studio is expecting config.json to be in training_checkpoints,

    io = ZarrBackend()
    with torch.no_grad():
        io = deterministic([start_datetime.isoformat()], n_steps, model, initial_data, io, 
        # variables_list=variables
        )

    ds = xr.open_zarr(io.root.store)
    final_datetime = start_datetime + timedelta(hours=6 * n_steps)
    final_timestamp = np.datetime64(final_datetime)
    final_ds = _normalize_valid_times(ds, final_timestamp)

    if variables is not None:
        final_ds = final_ds[variables]

    output_path.parent.mkdir(parents=True, exist_ok=True)
    final_ds.to_netcdf(output_path, mode="w", format="NETCDF4")
    

In [47]:
def main(cli_args: Sequence[str] | None = None, config: InferenceConfig | None = None) -> None:
    """Run inference using CLI args or a provided configuration.

    When ``config`` is omitted, argparse consumes ``cli_args`` (or ``sys.argv``)
    and fills in any missing values from ``DEFAULT_CONFIG``. To hard-code
    settings for batch runs without passing flags, edit ``DEFAULT_CONFIG`` and
    invoke the script without arguments.
    """

    cfg = config or _parse_args(DEFAULT_CONFIG, cli_args)
    start_dt = datetime.fromisoformat(cfg.start)

    _run_inference(
        start_datetime=start_dt,
        n_steps=cfg.steps,
        checkpoint_dir=cfg.checkpoint_dir,
        checkpoint_name=cfg.checkpoint_name,
        init_data_path=cfg.init_data,
        output_path=cfg.output,
        variables=cfg.variables,
        ema=cfg.ema,
    )

In [48]:
if __name__ == "__main__":
    main([])

2025-12-10 15:38:09.590 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-12-10 15:38:09.590 | INFO     | earth2studio.run:deterministic:82 - Inference device: cpu
2025-12-10 15:38:10.098 | SUCCESS  | earth2studio.run:deterministic:106 - Fetched data from DataArrayFile
2025-12-10 15:38:10.180 | INFO     | earth2studio.run:deterministic:136 - Inference starting!


Running inference: 100%|██████████| 5/5 [09:20<00:00, 112.00s/it]


2025-12-10 15:47:30.203 | SUCCESS  | earth2studio.run:deterministic:146 - Inference complete


In [40]:
# Checking initialization file contents (truth at valid time is at time index 2)
fp='/projectnb/eb-general/wade/sfno/inference_runs/Ian/Initialize_data/Initialize_2019_09_03T00_nsteps4.nc'
ds = xr.open_dataset(fp)
ds

<xarray.Dataset> Size: 615MB
Dimensions:    (time: 2, variable: 74, lat: 721, lon: 1440)
Coordinates:
  * time       (time) datetime64[ns] 16B 2019-09-03 2019-09-04
  * variable   (variable) <U5 1kB 'u10m' 'v10m' 'u100m' ... 'q925' 'q1000'
  * lat        (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon        (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
    grid_type  <U11 44B ...
Data variables:
    fields     (time, variable, lat, lon) float32 615MB ...

In [49]:
# Checking output file contents
fp = '/projectnb/eb-general/wade/sfno/inference_runs/sandbox/best_ckpt_2019_09_04T00_nsteps4.nc'
ds = xr.open_dataset(fp)
ds


<xarray.Dataset> Size: 307MB
Dimensions:     (valid_time: 1, lat: 721, lon: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 8B 2019-09-04
  * lat         (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon         (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables: (12/74)
    t1000       (valid_time, lat, lon) float32 4MB ...
    v1000       (valid_time, lat, lon) float32 4MB ...
    v200        (valid_time, lat, lon) float32 4MB ...
    t150        (valid_time, lat, lon) float32 4MB ...
    q700        (valid_time, lat, lon) float32 4MB ...
    u100        (valid_time, lat, lon) float32 4MB ...
    ...          ...
    u500        (valid_time, lat, lon) float32 4MB ...
    z850        (valid_time, lat, lon) float32 4MB ...
    z150        (valid_time, lat, lon) float32 4MB ...
    u400        (valid_time, lat, lon) float32 4MB ...
    q200        (valid_time, lat, lon) float32 4MB ...
    u200        (valid_time, lat, lon) float32 4MB ...
Attributes:
    initial_time:  2019-09-03T00:00:00.000000000

In [ ]:
# Plot var from output file and from the truth file to compare
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

var = 't2m'

# old version of sandbox code:


In [33]:
# Edit these defaults to set preferred settings when launching the
# script without CLI arguments (e.g., from a batch script).
DEFAULT_CONFIG = InferenceConfig(
    start="2019-09-03T00:00:00",
    steps=4,
    init_data=Path("/projectnb/eb-general/wade/sfno/inference_runs/Ian/Initialize_data/Initialize_2019_09_03T00_nsteps4.nc"),
    checkpoint_dir=Path("/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/"), # TODO CHECK IF THIS PATH NEEDS TO HAVE ADDITIONAL DIRECTORY TO THE MULTISTEP OR NON MULTISTEP DIR + seed + training_checkpoints
    checkpoint_name="ckpt_mp0_epoch1.tar", # for now, the code will just grab best ckpt, not this one.
    output=Path("/projectnb/eb-general/wade/sfno/inference_runs/sandbox/best_ckpt_2019_09_04T00_nsteps4.nc"), # todo make this automated to just take the directory, not the full path+name
    variables=['msl'],
    ema=False,
)


In [34]:
def _parse_args(defaults: InferenceConfig, cli_args: Sequence[str] | None = None) -> InferenceConfig:
    """Parse CLI arguments while honoring editable in-file defaults.

    Parameters
    ----------
    defaults : InferenceConfig
        Baseline values to use when a flag is omitted. Edit ``DEFAULT_CONFIG``
        to change these defaults without typing flags.
    cli_args : Sequence[str] | None
        Argument vector to parse. When ``None`` (the common case), argparse
        inspects ``sys.argv``. Passing an explicit sequence is useful for
        programmatic invocation.
    """

    parser = argparse.ArgumentParser(description="Run SFNO inference with earth2studio 0.10.x")
    parser.add_argument(
        "--start",
        default=defaults.start,
        help="ISO8601 start datetime for the forecast (e.g. 2019-03-22T00:00:00)",
    )
    parser.add_argument("--steps", type=int, default=defaults.steps, help="Number of 6-hour steps to forecast")
    parser.add_argument("--init-data", default=str(defaults.init_data), help="Path to the preprocessed initial state NetCDF")
    parser.add_argument("--checkpoint-dir", default=str(defaults.checkpoint_dir), help="Directory containing the SFNO checkpoints")
    parser.add_argument(
        "--checkpoint-name",
        default=defaults.checkpoint_name,
        help="Checkpoint file name inside the checkpoint directory",
    )
    parser.add_argument(
        "--output",
        default=str(defaults.output),
        help="Output NetCDF path for the final forecast timestep",
    )
    parser.add_argument(
        "--variables",
        nargs="+",
        default=defaults.variables,
        help="Optional list of variables to keep (defaults to all variables)",
    )
    parser.add_argument(
        "--ema",
        action="store_true",
        default=defaults.ema,
        help="Load EMA weights instead of the standard checkpoint",
    )

    args = parser.parse_args(args=cli_args)

    ### ADDITION:
    args, unknown = parser.parse_known_args(args=cli_args)
    ###

    return InferenceConfig(
        start=args.start,
        steps=args.steps,
        init_data=Path(args.init_data),
        checkpoint_dir=Path(args.checkpoint_dir),
        checkpoint_name=args.checkpoint_name,
        output=Path(args.output),
        variables=args.variables,
        ema=args.ema,
    )

In [35]:
def _normalize_valid_times(ds: xr.Dataset, final_timestamp: np.datetime64) -> xr.Dataset:
    """Align the earth2studio ``time`` and ``lead_time`` axes to usable timestamps.

    earth2studio returns forecasts with two axes: the initialization time (``time``)
    and the offsets from that initialization (``lead_time``). To mirror the legacy
    inference example, we compute the actual verification timestamps (``valid_time``),
    attach them as coordinates, and extract the requested final forecast hour.

    Parameters
    ----------
    ds : xr.Dataset
        Dataset returned by earth2studio after running inference.
    final_timestamp : np.datetime64
        Target forecast verification time to extract.
    """

    ds = ds.copy()
    ds["time"] = ds["time"].astype("datetime64[ns]")
    base_time = ds["time"].values
    lead_timedelta = ds["lead_time"].values.astype("timedelta64[ns]")
    valid_timesteps = (base_time[:, None] + lead_timedelta[None, :]).flatten()

    # Preserve the initial time as metadata while collapsing the time dimension.
    initial_time = str(ds["time"].values[0])
    ds = ds.isel(time=0).drop_vars("time")
    ds.attrs["initial_time"] = initial_time

    ds = ds.rename({"lead_time": "valid_time"})
    ds = ds.assign_coords(valid_time=(("valid_time",), valid_timesteps))

    if final_timestamp not in ds["valid_time"].values:
        raise ValueError(
            f"Requested final datetime {final_timestamp} not found in forecast valid times: {ds['valid_time'].values!r}"
        )

    return ds.sel(valid_time=[final_timestamp])


In [36]:
def _load_model(package_dir: Path, 
                # checkpoint_name: str, ema: bool
        ) -> torch.nn.Module:
    model_package = Package(str(package_dir), cache=False)
    return SFNO.load_model(model_package, 
    # checkpoint_name=checkpoint_name, EMA=ema
    )



In [37]:
def _run_inference(
    start_datetime: datetime,
    n_steps: int,
    checkpoint_dir: Path,
    checkpoint_name: str,
    init_data_path: Path,
    output_path: Path,
    variables: Iterable[str] | None,
    ema: bool,
) -> None:
    if not init_data_path.exists():
        raise FileNotFoundError(
            f"Initial data file {init_data_path} not found. Generate it before running inference."
        )

    initial_data = DataArrayFile(str(init_data_path))
    model = _load_model(checkpoint_dir, 
    # checkpoint_name,  ema
    )
    # Since earth2studio is expecting config.json to be in training_checkpoints,

    io = ZarrBackend()
    with torch.no_grad():
        io = deterministic([start_datetime.isoformat()], n_steps, model, initial_data, io, 
        # variables_list=variables
        )

    ds = xr.open_zarr(io.root.store)
    final_datetime = start_datetime + timedelta(hours=6 * n_steps)
    final_timestamp = np.datetime64(final_datetime)
    final_ds = _normalize_valid_times(ds, final_timestamp)

    if variables is not None:
        final_ds = final_ds[variables]

    output_path.parent.mkdir(parents=True, exist_ok=True)
    final_ds.to_netcdf(output_path, mode="w", format="NETCDF4")




In [38]:
def main(cli_args: Sequence[str] | None = None, config: InferenceConfig | None = None) -> None:
    """Run inference using CLI args or a provided configuration.

    When ``config`` is omitted, argparse consumes ``cli_args`` (or ``sys.argv``)
    and fills in any missing values from ``DEFAULT_CONFIG``. To hard-code
    settings for batch runs without passing flags, edit ``DEFAULT_CONFIG`` and
    invoke the script without arguments.
    """

    cfg = config or _parse_args(DEFAULT_CONFIG, cli_args)
    start_dt = datetime.fromisoformat(cfg.start)

    _run_inference(
        start_datetime=start_dt,
        n_steps=cfg.steps,
        checkpoint_dir=cfg.checkpoint_dir,
        checkpoint_name=cfg.checkpoint_name,
        init_data_path=cfg.init_data,
        output_path=cfg.output,
        variables=cfg.variables,
        ema=cfg.ema,
    )

In [39]:
if __name__ == "__main__":
    main([])


2025-12-10 14:58:34.958 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-12-10 14:58:34.958 | INFO     | earth2studio.run:deterministic:82 - Inference device: cpu
2025-12-10 14:58:35.519 | SUCCESS  | earth2studio.run:deterministic:106 - Fetched data from DataArrayFile
2025-12-10 14:58:35.654 | INFO     | earth2studio.run:deterministic:136 - Inference starting!


Running inference:  67%|██████▋   | 14/21 [34:21<17:10, 147.26s/it]


KeyboardInterrupt: 

# SCRIPT: (before edits)



In [ ]:
# """Example inference runner compatible with earth2studio 0.10.x.

# This script mirrors the workflow from the legacy
# ``inference/old_package_verisons/Inference_Example.py`` but updates the
# imports and I/O handling to align with the newer earth2studio API. It loads
# an SFNO checkpoint, runs deterministic inference starting from a provided
# initial state, and writes the requested variables for the final forecast time
# step to NetCDF.
# """
# from __future__ import annotations

# import argparse
# from dataclasses import dataclass
# from datetime import datetime, timedelta
# from pathlib import Path
# import sys
# from typing import Iterable, Sequence

# import numpy as np
# import torch
# import xarray as xr
# from earth2studio.data import DataArrayFile
# from earth2studio.io import ZarrBackend
# from earth2studio.models.auto import Package

# # # start with basic inference -- no edits to backend code 
# # # - using best ckpt and save all vars#

# # # TODO ONCE RUNNING INFERENCE - CHECK HOW THESE FUNCTIONS HAVE CHANGED IN THE NEW API, AND WHETHER OUR EDITS NEED TO BE CHANGED 
# # from deterministic_update import deterministic # vars -- less important
# # from SFNO_update import SFNO  # checkpoint access -- most important
# # take really diligent notes -- specific package versions
# # - get a yaml of the env with versions!!!


# @dataclass
# class InferenceConfig:
#     """Configuration container for running the inference script."""

#     start: str
#     steps: int
#     init_data: Path
#     checkpoint_dir: Path
#     checkpoint_name: str
#     output: Path
#     variables: list[str] | None
#     ema: bool


# # Edit these defaults to set preferred settings when launching the
# # script without CLI arguments (e.g., from a batch script).
# DEFAULT_CONFIG = InferenceConfig(
#     start="2019-03-22T00:00:00",
#     steps=12,
#     init_data=Path("/projectnb/eb-general/wade/sfno/inference_runs/Ian/Initialize_data/Initialize_2019_08_27T00_nsteps20.nc"),
#     checkpoint_dir=Path("/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/training_checkpoints/"), # TODO CHECK IF THIS PATH NEEDS TO HAVE ADDITIONAL DIRECTORY TO THE MULTISTEP OR NON MULTISTEP DIR + seed + training_checkpoints
#     checkpoint_name="ckpt_mp0_epoch1.tar",
#     output=Path("/projectnb/eb-general/wade/sfno/inference_runs/sandbox/"), # todo  fill in with desired output path?? or make it just output to dir + filename
#     variables=['msl'],
#     ema=False,
# )


# def _parse_args(defaults: InferenceConfig, cli_args: Sequence[str] | None = None) -> InferenceConfig:
#     """Parse CLI arguments while honoring editable in-file defaults.

#     Parameters
#     ----------
#     defaults : InferenceConfig
#         Baseline values to use when a flag is omitted. Edit ``DEFAULT_CONFIG``
#         to change these defaults without typing flags.
#     cli_args : Sequence[str] | None
#         Argument vector to parse. When ``None`` (the common case), argparse
#         inspects ``sys.argv``. Passing an explicit sequence is useful for
#         programmatic invocation.
#     """

#     parser = argparse.ArgumentParser(description="Run SFNO inference with earth2studio 0.10.x")
#     parser.add_argument(
#         "--start",
#         default=defaults.start,
#         help="ISO8601 start datetime for the forecast (e.g. 2019-03-22T00:00:00)",
#     )
#     parser.add_argument("--steps", type=int, default=defaults.steps, help="Number of 6-hour steps to forecast")
#     parser.add_argument("--init-data", default=str(defaults.init_data), help="Path to the preprocessed initial state NetCDF")
#     parser.add_argument("--checkpoint-dir", default=str(defaults.checkpoint_dir), help="Directory containing the SFNO checkpoints")
#     parser.add_argument(
#         "--checkpoint-name",
#         default=defaults.checkpoint_name,
#         help="Checkpoint file name inside the checkpoint directory",
#     )
#     parser.add_argument(
#         "--output",
#         default=str(defaults.output),
#         help="Output NetCDF path for the final forecast timestep",
#     )
#     parser.add_argument(
#         "--variables",
#         nargs="+",
#         default=defaults.variables,
#         help="Optional list of variables to keep (defaults to all variables)",
#     )
#     parser.add_argument(
#         "--ema",
#         action="store_true",
#         default=defaults.ema,
#         help="Load EMA weights instead of the standard checkpoint",
#     )

#     args = parser.parse_args(args=cli_args)
#     return InferenceConfig(
#         start=args.start,
#         steps=args.steps,
#         init_data=Path(args.init_data),
#         checkpoint_dir=Path(args.checkpoint_dir),
#         checkpoint_name=args.checkpoint_name,
#         output=Path(args.output),
#         variables=args.variables,
#         ema=args.ema,
#     )


# def _normalize_valid_times(ds: xr.Dataset, final_timestamp: np.datetime64) -> xr.Dataset:
#     """Align the earth2studio ``time`` and ``lead_time`` axes to usable timestamps.

#     earth2studio returns forecasts with two axes: the initialization time (``time``)
#     and the offsets from that initialization (``lead_time``). To mirror the legacy
#     inference example, we compute the actual verification timestamps (``valid_time``),
#     attach them as coordinates, and extract the requested final forecast hour.

#     Parameters
#     ----------
#     ds : xr.Dataset
#         Dataset returned by earth2studio after running inference.
#     final_timestamp : np.datetime64
#         Target forecast verification time to extract.
#     """

#     ds = ds.copy()
#     ds["time"] = ds["time"].astype("datetime64[ns]")
#     base_time = ds["time"].values
#     lead_timedelta = ds["lead_time"].values.astype("timedelta64[ns]")
#     valid_timesteps = (base_time[:, None] + lead_timedelta[None, :]).flatten()

#     # Preserve the initial time as metadata while collapsing the time dimension.
#     initial_time = str(ds["time"].values[0])
#     ds = ds.isel(time=0).drop_vars("time")
#     ds.attrs["initial_time"] = initial_time

#     ds = ds.rename({"lead_time": "valid_time"})
#     ds = ds.assign_coords(valid_time=(("valid_time",), valid_timesteps))

#     if final_timestamp not in ds["valid_time"].values:
#         raise ValueError(
#             f"Requested final datetime {final_timestamp} not found in forecast valid times: {ds['valid_time'].values!r}"
#         )

#     return ds.sel(valid_time=[final_timestamp])


# def _load_model(package_dir: Path, 
#                 # checkpoint_name: str, ema: bool
#         ) -> torch.nn.Module:
#     model_package = Package(str(package_dir), cache=False)
#     return SFNO.load_model(model_package, 
#     # checkpoint_name=checkpoint_name, EMA=ema
#     )


# def _run_inference(
#     start_datetime: datetime,
#     n_steps: int,
#     checkpoint_dir: Path,
#     checkpoint_name: str,
#     init_data_path: Path,
#     output_path: Path,
#     variables: Iterable[str] | None,
#     ema: bool,
# ) -> None:
#     if not init_data_path.exists():
#         raise FileNotFoundError(
#             f"Initial data file {init_data_path} not found. Generate it before running inference."
#         )

#     initial_data = DataArrayFile(str(init_data_path))
#     model = _load_model(checkpoint_dir, 
#     # checkpoint_name, 
#         ema)

#     io = ZarrBackend()
#     with torch.no_grad():
#         io = deterministic([start_datetime.isoformat()], n_steps, model, initial_data, io, 
#         # variables_list=variables
#         )

#     ds = xr.open_zarr(io.root.store)
#     final_datetime = start_datetime + timedelta(hours=6 * n_steps)
#     final_timestamp = np.datetime64(final_datetime)
#     final_ds = _normalize_valid_times(ds, final_timestamp)

#     if variables is not None:
#         final_ds = final_ds[variables]

#     output_path.parent.mkdir(parents=True, exist_ok=True)
#     final_ds.to_netcdf(output_path, mode="w", format="NETCDF4")



# def main(cli_args: Sequence[str] | None = None, config: InferenceConfig | None = None) -> None:
#     """Run inference using CLI args or a provided configuration.

#     When ``config`` is omitted, argparse consumes ``cli_args`` (or ``sys.argv``)
#     and fills in any missing values from ``DEFAULT_CONFIG``. To hard-code
#     settings for batch runs without passing flags, edit ``DEFAULT_CONFIG`` and
#     invoke the script without arguments.
#     """

#     cfg = config or _parse_args(DEFAULT_CONFIG, cli_args)
#     start_dt = datetime.fromisoformat(cfg.start)

#     _run_inference(
#         start_datetime=start_dt,
#         n_steps=cfg.steps,
#         checkpoint_dir=cfg.checkpoint_dir,
#         checkpoint_name=cfg.checkpoint_name,
#         init_data_path=cfg.init_data,
#         output_path=cfg.output,
#         variables=cfg.variables,
#         ema=cfg.ema,
#     )


# if __name__ == "__main__":
#     main()
